<a href="https://colab.research.google.com/github/sam-pitcher/system_activity_pipeline/blob/main/looker_system_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install looker_sdk

In [ ]:
pip install urllib3

In [ ]:
import looker_sdk
import urllib3
import csv
import os
import pprint as pp
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# if using Cloud Functions, highly recommend storing Looker API creds in Secret Manager
os.environ['LOOKERSDK_BASE_URL'] = 'ADD_YOUR_BASE_URL'
os.environ['LOOKERSDK_CLIENT_ID'] = 'ADD_YOUR_CLIENT_ID'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'ADD_YOUR_CLIENT_SECRET'
os.environ['LOOKERSDK_VERIFY_SSL']= 'False'
os.environ['LOOKERSDK_API_VERSION']= '4.0'
os.environ["LOOKERSDK_TIMEOUT"] = "120"

sdk = looker_sdk.init40()
print(sdk.me())

User(can={'show': True, 'index': True, 'show_details': True, 'index_details': True, 'sudo': False}, avatar_url='https://gravatar.lookercdn.com/avatar/dc540492ec307b9664f0bed96c36d291?s=156&d=blank', avatar_url_without_sizing='https://gravatar.lookercdn.com/avatar/dc540492ec307b9664f0bed96c36d291?d=blank', credentials_api3=[CredentialsApi3(can={}, id=75, client_id='mNs3WH6BXdfqv9s4Kq3K', created_at='2021-10-06T10:00:05.000+00:00', is_disabled=False, type='api3', url='https://localhost:19999/api/3.1/users/7405/credentials_api3/75')], credentials_email=CredentialsEmail(can={'show_password_reset_url': True}, created_at='2019-02-05T14:16:14.000+00:00', email='sam.pitcher@looker.com', forced_password_reset_at_next_login=False, is_disabled=False, logged_in_at='2022-02-03T13:48:54.000+00:00', password_reset_url='', type='email', url='https://localhost:19999/api/3.1/users/7405/credentials_email', user_url='https://localhost:19999/api/3.1/users/7405'), credentials_embed=[], credentials_google=No

In [ ]:
import json

# function takes:
# sys_act_model: this is the table in the currenct BigQuery System Activity. Forexample history, user, query
# limit: needs to be 'limit INTEGER' (this should be kept as a blank string unless testing)
# date_filter: the where claue for the date. Useful for only updating with yesterday's data eg: "where cast(created_at as date) > date_sub(current_date, interval 1 day)"

# function returns a json of the system activity data

def sys_act_data(sys_act_model, limit, date_filter):
  SQL = f"SELECT * FROM {sys_act_model} {date_filter} {limit}"
  print(f"SQL: {SQL}")
  sql_body = looker_sdk.models.SqlQueryCreate(connection_name = "looker__internal__analytics", sql = SQL)
  slug = sdk.create_sql_query(body=sql_body).slug
  result = sdk.run_sql_query(slug=slug, result_format="json")
  return json.loads(result)

In [ ]:
from google.cloud import bigquery
import os

# client can be used to interact with BigQuery (adding data, tables, datasets)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'looker-ps-emea-consultants.json'
client = bigquery.Client()

In [ ]:
PROJECT_NAME='YOUR_PROJECT_NAME'
DATASET_NAME='system_activity'

dataset = bigquery.Dataset(f'{PROJECT_NAME}.{DATASET_NAME}')
try:
  dataset = client.create_dataset(dataset)
except:
  print('dataset already exists')
# need to add exception rule

In [ ]:
from google.cloud.bigquery import table

# function used to create a table in BigQuery. It takes the first row of data and defines the schema with that row
# function takes:
# SCHEMA_DATA: a dict or json of the data going into the table

def create_bigquery_table(SCHEMA_DATA, DATASET_NAME, TABLE_NAME):
  dataset_ref = client.dataset(DATASET_NAME)
  table_ref = dataset_ref.table(TABLE_NAME)
  schema = []
  for column in SCHEMA_DATA[0]:
    if type(SCHEMA_DATA[0][column]) == int:
      field_type = 'int64'
    elif type(SCHEMA_DATA[0][column]) == float:
      field_type = 'float64'
    else:
      field_type = 'string'
    schema.append(
        bigquery.schema.SchemaField(
          name = column,
          field_type = field_type
        )
      )
  table = bigquery.table.Table(table_ref, schema=schema)
  table = client.create_table(table, exists_ok=True)
  return table

In [ ]:
# function used to add data to a table in BigQuery.
# function takes:
# SCHEMA_DATA: a dict or json of the data going into the table

def add_data_to_bigquery_table(SCHEMA_DATA, DATASET_NAME, TABLE_NAME):
  dataset_ref = client.dataset(DATASET_NAME)
  table_ref = dataset_ref.table(TABLE_NAME)
  table = client.get_table(table_ref)

  inserts = client.insert_rows_json(table, SCHEMA_DATA)
  return inserts

In [ ]:
PROJECT_NAME='YOUR_PROJECT_NAME'
DATASET_NAME='system_activity'

# this is the list of models available in the BigQuery System Activity Model already defined by Looker

# Note: group, source_query, source_query_rank and user_query_rank are unavailable to query via SQL and
# are therefore commented out. There is another way to get the group data defined below

sys_act_models = [
                  'history',
                  # 'group',
                  'group_user',
                  'look',
                  'query',
                  'result_maker',
                  'merge_query',
                  'merge_query_source_query',
                  # 'source_query',
                  'sql_query',
                  'user',
                  'user_facts',
                  'user_facts_role',
                  'role',
                  'permission_set',
                  'model_set',
                  'node'
                  # 'source_query_rank',
                  # 'user_query_rank'
                  
]

In [ ]:
# BUILD THE TABLES

for i in sys_act_models:
  print(f"Processing the {i} model.")
  r = sys_act_data(sys_act_model = i, date_filter = "", limit = "limit 30")
  print(r)
  TABLE_NAME = i
  try:
    create_bigquery_table(SCHEMA_DATA=r, DATASET_NAME=DATASET_NAME, TABLE_NAME=TABLE_NAME)
  except:
    pass

# GROUP TABLE
SCHEMA_DATA = [{'group_id': 1, 'group_name': 'name'}]
create_bigquery_table(SCHEMA_DATA=SCHEMA_DATA, DATASET_NAME=DATASET_NAME, TABLE_NAME='group')


Table(TableReference(DatasetReference('looker-ps-emea-consultants', 'system_activity'), 'group'))

In [ ]:
# ADD DATA EACH DAY TO THE TABLES

for i in sys_act_models:
  print(f"Processing the {i} model.")
  r = sys_act_data(
      sys_act_model = i,
      date_filter = "where cast(created_at as date) > date_sub(current_date, interval 1 day)",
      limit = ""
      )
  print(r)
  TABLE_NAME = i
  
  try:
    add_data_to_bigquery_table(SCHEMA_DATA=r, DATASET_NAME=DATASET_NAME, TABLE_NAME=TABLE_NAME)
  except:
    pass

# GROUP TABLE
groups = sdk.all_groups()
SCHEMA_DATA = []
for group in groups:
  SCHEMA_DATA.append({'group_id': group.id, 'group_name': group.name})

add_data_to_bigquery_table(
    SCHEMA_DATA=SCHEMA_DATA,
    DATASET_NAME=DATASET_NAME,
    TABLE_NAME='group')


[]